# Experiment 002: C++ SA Optimizer

Test the C++ SA optimizer from Jonathan Chan's kernel on our current best solution.
The strategy notes that previous experiments showed 0 improvement, but we should verify this.

In [1]:
import pandas as pd
import numpy as np
import os
import shutil

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

from shapely.geometry import Polygon

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        total += compute_score_for_n(df, n)
    return total

print("Functions defined")

Functions defined


In [2]:
# Load current best solution
df_best = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
original_score = compute_total_score(df_best)
print(f"Original score: {original_score:.6f}")

Original score: 70.659959


In [3]:
# Check if bbox3 binary exists and is executable
bbox3_path = '/home/code/external_data/saspav/bbox3'
if os.path.exists(bbox3_path):
    os.chmod(bbox3_path, 0o755)
    print(f"bbox3 binary found at {bbox3_path}")
    !{bbox3_path} --help 2>&1 | head -20
else:
    print("bbox3 binary not found")

bbox3 binary found at /home/code/external_data/saspav/bbox3
/home/code/external_data/saspav/bbox3: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /home/code/external_data/saspav/bbox3)


In [ ]:
# Copy the best solution to working directory
shutil.copy('/home/code/external_data/saspav/santa-2025.csv', '/home/code/experiments/002_cpp_optimizer/input.csv')
print("Copied input file")

In [ ]:
# Run bbox3 optimizer with limited iterations to test
# -n: number of iterations, -r: number of restarts
import subprocess
import time

start_time = time.time()
result = subprocess.run(
    [bbox3_path, '-n', '1000', '-r', '4'],
    cwd='/home/code/experiments/002_cpp_optimizer',
    capture_output=True,
    text=True,
    timeout=300  # 5 minute timeout
)
end_time = time.time()

print(f"Execution time: {end_time - start_time:.1f}s")
print("STDOUT:")
print(result.stdout[:2000] if result.stdout else "(empty)")
print("\nSTDERR:")
print(result.stderr[:1000] if result.stderr else "(empty)")

In [ ]:
# Check if output was generated
output_files = [f for f in os.listdir('/home/code/experiments/002_cpp_optimizer') if f.endswith('.csv')]
print(f"CSV files in directory: {output_files}")

# Check for any improvement
for f in output_files:
    if f != 'input.csv':
        df_opt = pd.read_csv(f'/home/code/experiments/002_cpp_optimizer/{f}')
        opt_score = compute_total_score(df_opt)
        improvement = original_score - opt_score
        print(f"{f}: score={opt_score:.6f}, improvement={improvement:.9f}")

In [ ]:
# bbox3 binary is incompatible with this system's GLIBC
# Let's try to compile the C++ optimizer from Jonathan Chan's kernel

# First, let's extract the C++ code from the notebook
import json

with open('/home/code/research/kernels/jonathanchan_santa25-ensemble-sa-fractional-translation/santa25-ensemble-sa-fractional-translation.ipynb', 'r') as f:
    nb = json.load(f)

# Find the cell with C++ code
for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if '#include' in source and 'fractional_translation' in source:
            # Save the C++ code
            with open('/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp', 'w') as cpp_file:
                cpp_file.write(source)
            print("Extracted C++ code")
            print(f"Code length: {len(source)} characters")
            break